In [1]:
#!pip install ipywidgets
#import ipywidgets as widgets
#from IPython.display import display, clear_output
#from digital_twin_ui import launch_ui

#launch_ui("use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json")


In [2]:
#!pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh



In [3]:
import random
from nibelungenbruecke.scripts.digital_twin_orchestrator.interface_test_DT import DigitalTwinInterface
import os



In [4]:
path = 'use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
# path = '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
    

In [5]:
dt = DigitalTwinInterface(path)


FileNotFoundError: Error: The file ../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json was not found!

In [ ]:
print(dt)

In [ ]:
dt.run_model()